https://ofai.github.io/million-post-corpus/ <br> **Posts** - Headline+Body, ID_User

In [1]:
ARTICLES_MIN_LENGTH = 300
ARTICLES_PER_AUTHOR = 250

- Tabelle - Posts
- Spalten: ID_User und Beitrag (Headline + Body)
- Entferne newlines und Links
- Wähle nur Posts mit einer Mindestlänge von 300
- Duplikate ignorieren

| columns | short posts |
| :-- | :-- |
| ![beitrag](img/headline_body.png)| ![too short post](img/short_body.png) |

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import sqlalchemy as sqla

con = sqlite3.connect("../../OneMillion/million_post_corpus/corpus.sqlite3", isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)

df = pd.read_sql_query("SELECT distinct ID_User, COALESCE(Headline, '') || COALESCE(Body, '') as Beitrag from Posts", con)

url = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)"

df = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r", url], value=[" "," ", "LINK"], regex=True)

df_minLength = df.loc[df['Beitrag'].apply(len) > ARTICLES_MIN_LENGTH]

In [3]:
## Anzahl Beiträge:
print(df_minLength.count())

ID_User    240863
Beitrag    240863
dtype: int64


In [4]:
## Anzahl der User
print(sum(df_minLength['ID_User'].value_counts()))

240863


In [5]:
## User mit nur einem Beitrag
print(sum(df_minLength['ID_User'].value_counts() == 1))

6233


In [6]:
## User mit mehr als 250 Beiträgen
print(sum(df_minLength['ID_User'].value_counts() > ARTICLES_PER_AUTHOR))

54


In [7]:
## neuer Datensatz ohne User weniger als 251 Beiträge
df_minArticles = df_minLength.groupby('ID_User').filter(lambda x : len(x) > ARTICLES_PER_AUTHOR)

In [8]:
df_minArticles

,ID_User,Beitrag
592,28526,Im großen war es sehr beeindrucktend. Man kann...
605,28526,Mir bleibt ein sehr junger Mann (etwa so um di...
787,6498,ich habe die 1980er damit verbracht in den Ost...
1121,6247,"Vor etwas mehr vielen vielen Jahren, so gegen ..."
1240,6247,Ein Sternehotel im Montafon.In den späten 80ig...
...,...,...
1009952,29722,"Hier wird etwas getan, was bei Gender auch ger..."
1009996,29722,"Andersdenkende einfach als ""Pigs"" also Schwein..."
1009998,29722,Das ist eine lupenreine Verschwörungstheorie: ...
1010126,29722,"Sie machen jetzt genau das, was ich im unteren..."


In [9]:
## ID_User - Anzahl passende Beiträge
df_minArticles['ID_User'].value_counts()

5854     731
25987    613
22876    595
30344    574
14587    512
26866    473
28816    457
20141    419
24984    417
20349    409
3788     406
18083    401
1795     399
15686    382
18446    375
24667    374
6247     367
19915    366
8765     363
15886    360
10627    352
16722    342
3105     338
6498     336
24758    333
11656    332
27393    331
12471    328
30661    328
29722    318
26532    312
2690     304
1330     300
18940    300
18984    299
21150    299
11582    297
24343    294
12887    289
11910    286
8446     285
29377    285
25582    282
12071    281
12023    275
8323     272
1560     270
7439     266
6169     264
27991    264
16328    259
28526    256
18006    254
28084    251
Name: ID_User, dtype: int64

In [10]:
import random

UserIDListe = df_minArticles.drop_duplicates(subset = ["ID_User"])['ID_User'].tolist()

ARTICLES_PER_AUTHOR_BSP = 250
AUTHORS = 20
IDS = random.sample(UserIDListe, AUTHORS)

In [11]:
## 20 Autoren die den Kriterien entsprechen:
print(*IDS, sep=', ')

16722, 16328, 18984, 27991, 26532, 11656, 18006, 28084, 1795, 3788, 12471, 18083, 11582, 25582, 12071, 1330, 30344, 8446, 7439, 6498


In [12]:
## Erstelle csv Datei für diese 20 Autoren
import os  
os.makedirs('data/users', exist_ok=True)
for x in range(AUTHORS):
    df_author = df_minArticles[df_minArticles['ID_User'] == IDS[x]].iloc[:ARTICLES_PER_AUTHOR,:]
    df_author.to_csv('data/users/' + str(IDS[x]) + '_posts.csv', index=False, columns=['Beitrag'])